# Loop Workflow Example 3

* High level approach to making a 3D model from just a bounding box and source files as input. (In Australia only for now. Documentation to come)
* This notebook uses example data provided by the Geological Survey of Western Australia.
* To run with your own data you will need to run the Utility 1 notebook to create an hjson config file and supply your own dtm model (service or geotif file in local coordinate system)

In [ ]:
#if not already installed:
#!conda install -c loop3d loopprojectfile map2loop loopstructural pyamg meshio 
#!conda install pyvista trame trame-vuetify trame-vtk

In [ ]:
# if pyvista keeps crashing the jupyter notebook (often in Windows), creating
# a very simple pyvista plotter before-hand often fixes this issue

import pyvista
pyvista.Plotter(window_size=[1,1]).show()

## Map2Loop

In [ ]:
import os
from map2loop.project import Project
from map2loop.m2l_enums import VerboseLevel
from map2loop.m2l_enums import Datatype
from map2loop.sampler import SamplerSpacing, SamplerDecimator
from map2loop.sorter import SorterUseHint, SorterUseNetworkX, SorterAgeBased, SorterAlpha
import time

from datetime import datetime
nowtime=datetime.now().isoformat(timespec='minutes')   
model_name=nowtime.replace(":","-").replace("T","-")
loop_project_filename = os.path.join(model_name, "local_source.loop3d")

t0 = time.time()

# Specify the boundary of the region of interest in the appropriate projection coordinates
bounding_box = {
    "minx": 520000,
    "miny": 7490000,
    "maxx": 550000,
    "maxy": 7510000,
    "base": -3200,
    "top": 1200,
}
# Initialise the project with the shapefiles, dtm, config file
# output locations and projection to work in
proj = Project( 
    geology_filename = "./source_data/geol_clip.shp",
    fault_filename = "./source_data/faults_clip.shp",
    structure_filename = "./source_data/structure_clip.shp",
    dtm_filename = './source_data/dtm_rp.tif',
    config_filename = './source_data/example.json',
    clut_filename = './source_data/500kibg_colours.csv',
    verbose_level = VerboseLevel.NONE,
    tmp_path = model_name,
    working_projection = "EPSG:28350",
    bounding_box = bounding_box,
    loop_project_filename = loop_project_filename,
    overwrite_loopprojectfile = True
)

# Remove faults less than 5km
proj.set_minimum_fault_length(5000.0)

# Set sampling distance for geology and fault maps to 200m
proj.set_sampler(Datatype.GEOLOGY, SamplerSpacing(200.0))
proj.set_sampler(Datatype.FAULT, SamplerSpacing(200.0))

# Set to only take every second orientation observation (0 or 1 means take all observations)
proj.set_sampler(Datatype.STRUCTURE, SamplerDecimator(2))

# Set what text is expected for intrusions (contained within the description field)
proj.map_data.config.geology_config["intrusive_text"] = "mafic intrusive"

# Set specific layers from the geology map to be ignored (commonly "cover" or "water")
proj.set_ignore_codes(["cover", "Fortescue_Group", "A_FO_od"])


# Specify which stratigraphic columns sorter to use, other options are
# (SorterAlpha, SorterAgeBased, SorterUseHint, SorterUseNetworkX, SorterMaximiseContacts, SorterObservationProjections)
proj.set_sorter(SorterAlpha())

# Or you can run map2loop and pre-specify the stratigraphic column
column = [
    # youngest
    'Turee_Creek_Group',
    'Boolgeeda_Iron_Formation',
    'Woongarra_Rhyolite',
    'Weeli_Wolli_Formation',
    'Brockman_Iron_Formation',
    'Mount_McRae_Shale_and_Mount_Sylvia_Formation',
    'Wittenoom_Formation',
    'Marra_Mamba_Iron_Formation',
    'Jeerinah_Formation',
    'Bunjinah_Formation',
    'Pyradie_Formation',
    'Fortescue_Group',
    # oldest
]
proj.run_all(user_defined_stratigraphic_column=column)

# Or you can get map2loop to run all column sorting algorithms it has and takes the one
# that has the longest total basal contact length
#proj.run_all(take_best=True)
# proj.run_all()

t1 = time.time()

In [ ]:
# Draw overlay of point data on geology map
# (options are 'basal_contacts', contacts','orientations','faults')
proj.draw_geology_map(overlay="basal_contacts")

In [ ]:
# Extract estimate of the stratigraphic column
proj.stratigraphic_column.column

## Loop Structural

In [ ]:
import LoopProjectFile as LPF
import LoopStructural
from LoopStructural.modelling.input.project_file import LoopProjectfileProcessor as LPFProcessor

# LoopStructural Modelling
t2 = time.time()
# Set parameters for fault and foliation 
 # Alternate 'interpolatortype':'PLI',
fault_params = {
    'interpolatortype':'FDI',
    'nelements':1e4,
}
foliation_params = {
    'interpolatortype':'FDI' ,
    'nelements':1e5,  # how many tetras/voxels
    'regularisation':25,
}
projFile = LPF.ProjectFile(loop_project_filename)
processedData = LPFProcessor(projFile)
#processedData.foliation_properties['sg'] = foliation_params
#processedData.fault_properties['interpolatortype'] = fault_params['interpolatortype']
#processedData.fault_properties['nelements'] = fault_params['nelements']

model = LoopStructural.GeologicalModel.from_processor(processedData)
#model.nsteps=np.array([200,200,50])
model.update()

t3 = time.time()

## View 3D Data from Loop Structural

In [ ]:
t4 = time.time()

# Create viewer for Loop Structural output
from LoopStructural.visualisation import Loop3DView
view = Loop3DView(model, window_size=[800,600])
view.plot_model_surfaces(fault_colour="red")

# Add dtm method not currently implimented but will be coming soon
#view.add_dtm()

# Set inital camera position and orientation
view.camera_position = "xz"
view.camera.elevation = 40

view.show(jupyter_backend='client')

t5 = time.time()

## Elapsed Time

In [ ]:
# Print element and total processing time
m2l_time = t1-t0
ls_time = t3-t2
viewer_time = t5-t4
total = t5-t0

m2l_string = f"{m2l_time} sec" if m2l_time < 60 else f"{m2l_time/60.0} min"
ls_string = f"{ls_time} sec" if ls_time < 60 else f"{ls_time/60.0} min"
viewer_string = f"{viewer_time} sec" if viewer_time < 60 else f"{viewer_time/60.0} min"
total_string = f"{total} sec" if total < 60 else f"{total/60.0} min"
print(f" Map2loop {m2l_string}\n LoopStructural {ls_string}\n Viewer {viewer_string}\n Total {total_string}")